In [8]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
from scipy import sparse
from sklearn.metrics.pairwise import linear_kernel
from shiny import App, render, ui


In [9]:
#titles dataset from https://www.kaggle.com/datasets/victorsoeiro/rakuten-tv-dramas-and-movies/code?select=credits.csv
titles=pd.read_csv('/Users/YumaryVasquez/Desktop/data_science.nosync/viki/titles.csv')

In [10]:
tfidf = TfidfVectorizer(stop_words='english')
titles['genres'] = titles['genres'].fillna('')
tfidf_matrix = tfidf.fit_transform(titles['genres'])
indices = pd.Series(titles.index, index=titles['title']).drop_duplicates()
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [11]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return titles['title'].iloc[movie_indices]

In [12]:
app_ui = ui.page_fluid(
    ui.input_text("x", "Text input", placeholder="Enter Name of Show"),
    ui.output_text_verbatim("txt"),
)

In [13]:
def server(input, output, session):
    @output
    @render.text
    def txt():
        return(get_recommendations(input.x()))
        #return f'x: "{input.x()}"'



In [14]:
app = App(app_ui, server, debug=True)

In [66]:
#get_recommendations('Strong Woman Do Bong Soon')

273                    New World
1235             Beautiful World
797                Ashes of Love
503                  Oh My Ghost
601     My Heart Twinkle Twinkle
1449                  Year Beast
1699          The Second Husband
1171                        Item
1595                 Raging Fire
512             The Senior Class
Name: title, dtype: object